In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import random as rnd
import math
from pytrends.request import TrendReq
from seaborn import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, fbeta_score, classification_report

In [2]:
pytrends = TrendReq(hl='en-US', tz=360)

kw_list = ["Tesla"] 

pytrends.build_payload(kw_list, timeframe='2021-03-31 2021-06-29', geo='US')

Trendsdf = pytrends.interest_over_time()
Trendsdf = Trendsdf.rename(columns={'Tesla': 'Previous_Search_Interest'})
#Trendsdf.reset_index(inplace=True, drop=True)
Trendsdf.shape
Trendsdf

,Previous_Search_Interest,isPartial
date,,
2021-03-31,74,False
2021-04-01,80,False
2021-04-02,74,False
2021-04-03,64,False
2021-04-04,59,False
...,...,...
2021-06-25,70,False
2021-06-26,69,False
2021-06-27,63,False


In [3]:
stockPricePrevious = yf.download("TSLA", start="2021-03-31", end="2021-06-29")
stockPricePrevious = stockPricePrevious.rename(columns={'Open': 'Previous_Open', 'Close': 'Previous_Close','High': 'Previous_High','Volume': 'Previous_Volume','Low': 'Previous_Low', 'Adj Close':'Previous_Adj_Close'})
stockPricePrevious

mergedPriceTrends = pd.concat([stockPricePrevious, Trendsdf], axis=1)
mergedPriceTrends.reset_index(inplace=False, drop=True)
mergedPriceTrends.dropna(inplace=True)
#mergedPriceTrends.reset_index()


#print(mergedPriceTrends)
stockPrice = yf.download("TSLA", start="2021-04-01", end="2021-06-30")
#stockPrice.reset_index(inplace=True, drop=True)
#print(stockPrice)
mergedPrice = pd.concat([mergedPriceTrends, stockPrice], axis=1)
#mergedPrice['Previous_Open', 'Previous_High','Previous_Low','Previous_Close', 'Previous_Adj_Close', 'Previous_Volume'] = mergedPrice['Previous_Open', 'Previous_High','Previous_Low','Previous_Close', 'Previous_Adj_Close', 'Previous_Volume'].shift(1)
mergedPrice['Previous_Open'] = mergedPrice['Previous_Open'].shift(1)
mergedPrice['Previous_High'] = mergedPrice['Previous_High'].shift(1)
mergedPrice['Previous_Low'] = mergedPrice['Previous_Low'].shift(1)
mergedPrice['Previous_Close'] = mergedPrice['Previous_Close'].shift(1)
mergedPrice['Previous_Adj_Close'] = mergedPrice['Previous_Adj_Close'].shift(1)
mergedPrice['Previous_Volume'] = mergedPrice['Previous_Volume'].shift(1)
mergedPrice['Previous_Search_Interest'] = mergedPrice['Previous_Search_Interest'].shift(1)
mergedPrice.dropna(inplace=True)
#mergedPrice.set_index('isPartial', drop=False)
mergedPrice
#mergedPriceTrends

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Adj_Close,Previous_Volume,Previous_Search_Interest,isPartial,Open,High,Low,Close,Adj Close,Volume
2021-04-01,646.619995,672.000000,641.109985,667.929993,667.929993,33337300.0,74.0,False,688.369995,692.419983,659.419983,661.750000,661.750000,35298400.0
2021-04-05,688.369995,692.419983,659.419983,661.750000,661.750000,35298400.0,80.0,False,707.710022,708.159973,684.700012,691.049988,691.049988,41842800.0
2021-04-06,707.710022,708.159973,684.700012,691.049988,691.049988,41842800.0,78.0,False,690.299988,696.549988,681.369995,691.619995,691.619995,28271800.0
2021-04-07,690.299988,696.549988,681.369995,691.619995,691.619995,28271800.0,76.0,False,687.000000,691.380005,667.840027,670.969971,670.969971,26309400.0
2021-04-08,687.000000,691.380005,667.840027,670.969971,670.969971,26309400.0,66.0,False,677.380005,689.549988,671.650024,683.799988,683.799988,23924300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-22,624.479980,631.390015,608.880005,620.830017,620.830017,24812700.0,64.0,False,618.250000,628.570007,615.500000,623.710022,623.710022,19158900.0
2021-06-23,618.250000,628.570007,615.500000,623.710022,623.710022,19158900.0,64.0,False,632.000000,657.200012,630.039978,656.570007,656.570007,31099200.0
2021-06-24,632.000000,657.200012,630.039978,656.570007,656.570007,31099200.0,70.0,False,674.989990,697.619995,667.609985,679.820007,679.820007,45982400.0
2021-06-25,674.989990,697.619995,667.609985,679.820007,679.820007,45982400.0,79.0,False,689.580017,693.809998,668.700012,671.869995,671.869995,32496700.0


In [4]:
mergedPrice.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 61 entries, 2021-04-01 to 2021-06-28
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Previous_Open             61 non-null     float64
 1   Previous_High             61 non-null     float64
 2   Previous_Low              61 non-null     float64
 3   Previous_Close            61 non-null     float64
 4   Previous_Adj_Close        61 non-null     float64
 5   Previous_Volume           61 non-null     float64
 6   Previous_Search_Interest  61 non-null     float64
 7   isPartial                 61 non-null     object 
 8   Open                      61 non-null     float64
 9   High                      61 non-null     float64
 10  Low                       61 non-null     float64
 11  Close                     61 non-null     float64
 12  Adj Close                 61 non-null     float64
 13  Volume                    61 non-null     float

In [5]:
columns = ["Previous_Open", "Previous_High", "Previous_Low", "Previous_Close", "Previous_Volume", "Previous_Search_Interest", "Close"]
mergedPrice = mergedPrice[columns]
mergedPrice

,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Volume,Previous_Search_Interest,Close
2021-04-01,646.619995,672.000000,641.109985,667.929993,33337300.0,74.0,661.750000
2021-04-05,688.369995,692.419983,659.419983,661.750000,35298400.0,80.0,691.049988
2021-04-06,707.710022,708.159973,684.700012,691.049988,41842800.0,78.0,691.619995
2021-04-07,690.299988,696.549988,681.369995,691.619995,28271800.0,76.0,670.969971
2021-04-08,687.000000,691.380005,667.840027,670.969971,26309400.0,66.0,683.799988
...,...,...,...,...,...,...,...
2021-06-22,624.479980,631.390015,608.880005,620.830017,24812700.0,64.0,623.710022
2021-06-23,618.250000,628.570007,615.500000,623.710022,19158900.0,64.0,656.570007
2021-06-24,632.000000,657.200012,630.039978,656.570007,31099200.0,70.0,679.820007
2021-06-25,674.989990,697.619995,667.609985,679.820007,45982400.0,79.0,671.869995


In [6]:
features = list(mergedPrice.columns)
features.remove("Close")
target = "Close"

X = mergedPrice[features]
y = mergedPrice[target]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)

In [8]:
lr = LinearRegression()
lr

LinearRegression()

In [9]:
lr.fit(X_train, y_train)

LinearRegression()

In [10]:
lr.score(X_train, y_train)

0.8991402863681097

In [11]:
lr.score(X_test, y_test)

0.696338058422777

In [12]:
def printMetrics(test, predictions):
    print(f"Score: {explained_variance_score(test, predictions):.2f}")
    print(f"MAE: {mean_absolute_error(test, predictions):.2f}")
    print(f"RMSE: {math.sqrt(mean_squared_error(test, predictions)):.2f}")
    print(f"r2: {r2_score(test, predictions):.2f}")

In [13]:
predictions = lr.predict(X_test)
printMetrics(y_test, predictions)

Score: 0.70
MAE: 12.42
RMSE: 15.69
r2: 0.70


In [14]:
predictedPrice = pd.concat([X_test, y_test], axis=1)
predictedPrice = predictedPrice.rename(columns={'Close': 'Predicted_Close'})
predictedPrice

,Previous_Open,Previous_High,Previous_Low,Previous_Close,Previous_Volume,Previous_Search_Interest,Predicted_Close
2021-05-28,620.239990,631.130005,616.210022,630.849976,26370600.0,72.0,625.219971
2021-06-01,628.500000,635.590027,622.380005,625.219971,22737000.0,72.0,623.900024
2021-06-02,627.799988,633.799988,620.549988,623.900024,18084900.0,73.0,605.119995
2021-06-03,620.130005,623.359985,599.140015,605.119995,23302800.0,70.0,572.840027
2021-06-04,601.799988,604.549988,571.219971,572.840027,30111900.0,71.0,599.049988
2021-06-07,579.710022,600.609985,577.200012,599.049988,24036900.0,72.0,605.130005
2021-06-08,591.830017,610.000000,582.880005,605.130005,22543700.0,70.0,603.590027
2021-06-09,623.010010,623.090027,595.500000,603.590027,26053400.0,71.0,598.780029
2021-06-10,602.169983,611.789978,597.630005,598.780029,16584600.0,62.0,610.119995
2021-06-11,603.880005,616.590027,600.500000,610.119995,23919600.0,68.0,609.890015
